In [2]:
!wget https://dumps.wikimedia.org/enwiktionary/20230620/enwiktionary-20230620-pages-articles-multistream.xml.bz2

--2023-06-21 17:58:36--  https://dumps.wikimedia.org/enwiktionary/20230620/enwiktionary-20230620-pages-articles-multistream.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1309869940 (1.2G) [application/octet-stream]
Saving to: ‘enwiktionary-20230620-pages-articles-multistream.xml.bz2’

enwiktionary-202306 100%[===================>]   1.22G  3.69MB/s    in 5m 54s  

2023-06-21 18:04:31 (3.53 MB/s) - ‘enwiktionary-20230620-pages-articles-multistream.xml.bz2’ saved [1309869940/1309869940]



In [4]:
%%writefile pluck-ipa-hu.pl
#!/usr/bin/perl

use warnings;
use strict;
use utf8;

binmode(STDIN, ":utf8");
binmode(STDOUT, ":utf8");

my $title = '';
while(<>) {
	chomp;
	if(m!<title>([^<]+)</title>!) {
		$title = $1;
	}
  if($title =~ /^Wiktionary:/) {
    next;
  }
	if(m/\{\{IPA\|hu\|/) {
		if($title and $title ne '') {
			print "\n$title\n";
			$title = '';
		}
		print "$_\n";
	}
}

Overwriting pluck-ipa-hu.pl


In [5]:
!bzcat enwiktionary-20230620-pages-articles-multistream.xml.bz2 |perl pluck-ipa-hu.pl > hungarian-ipa-raw.txt

In [8]:
%%writefile pluck-ipa-hu-auto.pl
#!/usr/bin/perl

use warnings;
use strict;
use utf8;

binmode(STDIN, ":utf8");
binmode(STDOUT, ":utf8");

my $title = '';
while(<>) {
	chomp;
	if(m!<title>([^<]+)</title>!) {
		$title = $1;
	}
	if($title =~ /^Wiktionary:/) {
		next;
	}
	if(m/\{\{hu-IPA\|([^\}]+)\}\}/) {
		my $rep = $1;
		if($title and $title ne '') {
			print "$title\t$rep\n";
			$title = '';
			$rep = '';
		}
	} elsif(m/\{\{hu-IPA\}\}/) {
		print "$title\n";
	}
}

Overwriting pluck-ipa-hu-auto.pl


In [9]:
!bzcat enwiktionary-20230620-pages-articles-multistream.xml.bz2 |perl pluck-ipa-hu-auto.pl > hungarian-ipa-auto.txt